In [60]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial
from math import factorial
from scipy.linalg import solveh_banded
np.set_printoptions(precision=3, linewidth=200)
from scipy.special import roots_legendre
from scipy.integrate import quad
from scipy.misc import derivative
from scipy.stats import qmc

In [61]:
def scan_min_max(f, a, b):
    points = np.linspace(a, b, int((b - a) * 10))
    max = -np.inf
    min = np.inf
    for x in points:
        val = f(x)
        if val > max:
            max = val
        if val < min:
            min = val
    return min, max

In [62]:
def monte_carlo(f, a, b, mode, eps, verbose=False):
    # mode: std, geo, geo_sobol, geo_sobol_shuffle
    n = int( (3 * (b - a) / eps)**2 / 12) + 1
    min, max = scan_min_max(f, a, b)
    integration = quad(f, a, b)
    if verbose:
        print(n)
        print(f'{min=}, {max=}')
    match mode:
        case 'std':
            points = a + np.random.rand(n) * (b - a)
            res = 0
            for i in range(n):
                res += f(points[i])
            res *= (b - a) / n
            print('monte carlo res =', res)
            print('scipy integration result = ', integration[0])
            print('error =', np.abs(res - integration[0]))
            return res
        case 'geo':
            points_x = a + np.random.rand(n) * (b - a)
            points_y = np.random.rand(n) * (max - min) + min
        case 'geo_sobol':
            sampler = qmc.Sobol(d=2, scramble=False)
            points = sampler.random(n)
            points_x = [a + x[0] * (b - a) for x in points]
            points_y = [x[1] * (max - min) + min for x in points]
        case 'geo_sobol_shuffle':
            sampler = qmc.Sobol(d=2, scramble=True)
            points = sampler.random(n)
            points_x = [a + x[0] * (b - a) for x in points]
            points_y = [x[1] * (max - min) + min for x in points]
        case _:
            print(f'wrong mode {mode}')
            return 0
    res = 0
    for i in range(n):
        res += (f(points_x[i]) >= points_y[i])
    res *= (b - a) * (max - min) / n
    print('monte carlo res =', res)
    print('scipy integration result = ', integration[0])
    print('error =', np.abs(res - integration[0]))
    return res

In [63]:
np.random.seed(42)

In [64]:
eps = 0.01
monte_carlo(np.exp, -5, 5, 'std', eps, True)
monte_carlo(np.exp, -5, 5, 'geo', eps, True)
monte_carlo(np.exp, -5, 5, 'geo_sobol', eps, True)
monte_carlo(np.exp, -5, 5, 'geo_sobol_shuffle', eps, True)

750001
min=0.006737946999085467, max=148.4131591025766
monte carlo res = 148.45649063427012
scipy integration result =  148.40642115557753
error = 0.05006947869259193
750001
min=0.006737946999085467, max=148.4131591025766
monte carlo res = 148.4715220188566
scipy integration result =  148.40642115557753
error = 0.06510086327907061
750001
min=0.006737946999085467, max=148.4131591025766
monte carlo res = 148.33894579272592
scipy integration result =  148.40642115557753
error = 0.06747536285161004
750001
min=0.006737946999085467, max=148.4131591025766
monte carlo res = 148.32707329486348
scipy integration result =  148.40642115557753
error = 0.07934786071405142


148.32707329486348

In [65]:
def my_sin(x):
    return np.sin(x) + 1

In [66]:
eps = 0.01
monte_carlo(my_sin, -5, 8, 'std', eps, True)
monte_carlo(my_sin, -5, 8, 'geo', eps, True)
monte_carlo(my_sin, -5, 8, 'geo_sobol', eps, True)
monte_carlo(my_sin, -5, 8, 'geo_sobol_shuffle', eps, True)

1267501
min=4.05294107841403e-06, max=1.9998917427891376
monte carlo res = 13.432842096986972
scipy integration result =  13.42916221927184
error = 0.0036798777151325623
1267501
min=4.05294107841403e-06, max=1.9998917427891376
monte carlo res = 13.420179347724016
scipy integration result =  13.42916221927184
error = 0.008982871547823734
1267501
min=4.05294107841403e-06, max=1.9998917427891376
monte carlo res = 13.4293480563607
scipy integration result =  13.42916221927184
error = 0.00018583708886055206
1267501
min=4.05294107841403e-06, max=1.9998917427891376
monte carlo res = 13.428425032001302
scipy integration result =  13.42916221927184
error = 0.0007371872705377314


13.428425032001302

In [67]:
eps = 0.01
monte_carlo(Polynomial([1, 0, 1]), -3, 2, 'std', eps, True)
monte_carlo(Polynomial([1, 0, 1]), -3, 2, 'geo', eps, True)
monte_carlo(Polynomial([1, 0, 1]), -3, 2, 'geo_sobol', eps, True)
monte_carlo(Polynomial([1, 0, 1]), -3, 2, 'geo_sobol_shuffle', eps, True)

187501
min=1.0016659725114536, max=10.0
monte carlo res = 16.67497738477716
scipy integration result =  16.666666666666664
error = 0.008310718110497106
187501
min=1.0016659725114536, max=10.0
monte carlo res = 11.704970476447894
scipy integration result =  16.666666666666664
error = 4.96169619021877
187501
min=1.0016659725114536, max=10.0
monte carlo res = 11.654340120348458
scipy integration result =  16.666666666666664
error = 5.0123265463182065
187501
min=1.0016659725114536, max=10.0
monte carlo res = 11.6576994804688
scipy integration result =  16.666666666666664
error = 5.008967186197864


11.6576994804688